In [1]:
import pandas as pd
import src.nasty_score as nasty

### Read data and load models

In [2]:
data = pd.read_feather('data/nasty_data_2021.fea').fillna(-9999)
pitchers = pd.read_feather('data/pitcher_ids.fea')

In [3]:
sz_model = nasty.import_model('models/sz_model.pkl')
swing_model = nasty.import_model('models/swing_model.pkl')
swingmiss_model = nasty.import_model('models/swingmiss_model.pkl')

### Assign nasty scores

In [4]:
data = nasty.add_nasty_score(data, sz_model, swing_model, swingmiss_model)
data = data.merge(pitchers, on='pitcher', how='left')

### Tables

#### Table 1. Top 10 nastiest individual pitches of 2021

In [5]:
table_cols = ['player_name', 'game_date', 'pitch_type', 'release_speed',
              'release_spin_rate', 'nasty_score']

data[table_cols].sort_values(by='nasty_score', ascending=False).head(10)

,player_name,game_date,pitch_type,release_speed,release_spin_rate,nasty_score
615071,"Rodón, Carlos",2021-04-29,SL,87.6,2650,100.000000
669349,"Jackson, Luke",2021-04-14,SL,87.9,2645,99.139946
447118,"Newcomb, Sean",2021-06-11,FC,87.8,2796,96.099213
490196,"Rodón, Carlos",2021-05-31,SL,88.4,2608,95.616798
701411,"Glasnow, Tyler",2021-04-06,CU,83.8,3052,93.038109
590091,"Gallegos, Giovanny",2021-05-05,SL,84.2,2518,92.719582
321697,"Scott, Tanner",2021-07-17,SL,87.6,2783,92.298546
569145,"Romano, Jordan",2021-05-11,SL,89.3,2474,92.295853
684821,"Newcomb, Sean",2021-04-10,FC,87.1,2771,91.706993
475308,"Scott, Tanner",2021-06-04,SL,90.0,2817,91.281830


#### Table 2. Top 10 nastiest pitches of 2021

In [6]:
ranked_pitches = data.groupby(['player_name', 'pitch_type']).agg(
    avg_release_speed=('release_speed', 'mean'),
    avg_release_spin_rate=('release_spin_rate', 'mean'),
    pitch_count = pd.NamedAgg('nasty_score', 'count'),
    nasty_score = pd.NamedAgg('nasty_score', 'mean')).reset_index()

ranked_pitches.loc[ranked_pitches.pitch_count > 250]\
    .sort_values(by='nasty_score', ascending=False).head(10)

,player_name,pitch_type,avg_release_speed,avg_release_spin_rate,pitch_count,nasty_score
3218,"Treinen, Blake",SL,86.508625,2396.773893,429,41.686024
1528,"Jackson, Luke",SL,87.618769,2313.284615,650,41.677277
2473,"Pineda, Michael",SL,81.226923,1890.672065,494,41.257759
2079,"Melancon, Mark",KC,82.245179,2599.082645,363,41.057053
1056,"Gallegos, Giovanny",SL,85.600375,2482.917603,534,40.653336
1141,"Gibson, Kyle",SL,83.46645,2513.551948,462,39.548351
1389,"Hendrix, Ryan",SL,85.877441,2422.188552,297,38.397846
2598,"Rasmussen, Drew",SL,86.371237,2654.193548,372,38.210098
2227,"Musgrove, Joe",SL,84.071779,2783.440491,815,37.554077
1601,"Keller, Brad",SL,86.132518,2566.44621,818,37.539768


#### Table 3. Top 10 nastiest pitchers of 2021

In [7]:
ranked_pitches = data.groupby(['player_name']).agg(
    nasty_score = pd.NamedAgg('nasty_score', 'mean'),
    pitch_count = pd.NamedAgg('nasty_score', 'count')).reset_index()

ranked_pitches.loc[ranked_pitches.pitch_count > 400]\
    .sort_values(by='nasty_score', ascending=False).head(10)

,player_name,nasty_score,pitch_count
389,"Jackson, Luke",30.866423,1204
300,"Glasnow, Tyler",29.516430,1339
906,"deGrom, Jacob",28.688354,1226
733,"Scott, Tanner",28.480850,1032
32,"Anderson, Shaun",27.606268,437
565,"Musgrove, Joe",27.570084,2938
154,"Clase, Emmanuel",27.338806,1057
210,"Doval, Camilo",26.964865,499
514,"McGowin, Kyle",26.791321,517
603,"Pagán, Emilio",26.408237,1098


#### Table 4. Top 10 nastiest games of 2021

In [8]:
ranked_pitches = data.groupby(['player_name', 'game_date']).agg(
    nasty_score = pd.NamedAgg('nasty_score', 'mean'),
    pitch_count = pd.NamedAgg('nasty_score', 'count')).reset_index()

ranked_pitches.loc[ranked_pitches.pitch_count > 50]\
    .sort_values(by='nasty_score', ascending=False).head(10)

,player_name,game_date,nasty_score,pitch_count
13658,"Musgrove, Joe",2021-05-08,34.025532,90
13679,"Musgrove, Joe",2021-09-10,33.587414,106
21861,"deGrom, Jacob",2021-06-16,33.573277,51
21858,"deGrom, Jacob",2021-05-31,33.326897,70
13655,"Musgrove, Joe",2021-04-19,33.276825,96
15672,"Rasmussen, Drew",2021-08-06,33.246456,53
7209,"Glasnow, Tyler",2021-05-26,32.326454,102
7199,"Glasnow, Tyler",2021-04-01,32.309727,77
21860,"deGrom, Jacob",2021-06-11,32.156307,80
13672,"Musgrove, Joe",2021-07-29,32.123253,91
